In [1]:
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np

In [2]:
annotation = pd.read_csv('/home/icb/jonas.flor/gastrulation_atlas/data/GSE208680_cell_annotation.txt', sep='\t')
data = pd.read_csv('/home/icb/jonas.flor/gastrulation_atlas/data/GSE208680_raw_counts.csv', index_col='Name')
db = pd.read_csv('/home/icb/jonas.flor/gastrulation_atlas/data/mouse.v12.geneID.txt', sep='\t')[['gene_ID', 'gene_short_name']]

In [3]:
annotation

,Cell_barcode,nCount_RNA,nFeature_RNA,samples,percent.mt,seurat_clusters,Nat_Syn,Sample_Name,cluster_annotation,Unnamed: 9
0,AAACCCAAGACTAGAT-7,5315,2385,7,4.929445,0,Synthetic,Cdx2-10d-2pooled-r2,Amnion,NaN
1,AAACCCAAGACTAGAT-8,5105,2369,8,4.955926,0,Synthetic,Cdx2-10d-2pooled-r1,Amnion,NaN
2,AAACCCAAGTCTGGAG-8,11979,3915,8,3.364221,0,Synthetic,Cdx2-10d-2pooled-r1,Amnion,NaN
3,AAACCCAGTAATACCC-7,10775,3583,7,4.232019,0,Synthetic,Cdx2-10d-2pooled-r2,Amnion,NaN
4,AAACCCAGTAATACCC-8,10023,3448,8,4.379926,0,Synthetic,Cdx2-10d-2pooled-r1,Amnion,NaN
...,...,...,...,...,...,...,...,...,...,...
67600,TTCACCGTCCCGAACG-11,2723,1709,11,1.689313,23,Synthetic,TSCbl-2pooled,Extra-Embryonic endoderm,NaN
67601,TTCTAACAGAAGTGTT-11,824,665,11,4.126214,23,Synthetic,TSCbl-2pooled,Extra-Embryonic endoderm,NaN
67602,TTCTAACTCATGCTAG-7,2616,1700,7,7.415902,23,Synthetic,Cdx2-10d-2pooled-r2,Extra-Embryonic endoderm,NaN
67603,TTGGGTAAGTCTAGCT-11,1585,1116,11,0.189274,23,Synthetic,TSCbl-2pooled,Extra-Embryonic endoderm,NaN


In [15]:
adata = ad.AnnData(data.transpose())
adata = adata[adata.obs_names.isin(list(annotation['Cell_barcode'])), :]

adata.obs['Nat_Synth'] = pd.Categorical(annotation['Nat_Syn'])
adata.obs['embryo_id'] = pd.Categorical(annotation['Sample_Name'])
adata.obs['celltype'] = pd.Categorical(annotation['cluster_annotation'])
adata.obs['stage'] = pd.Categorical(['E8.5' for x in range(adata.n_obs)])

adata.obs.stage.unique()

/tmp/ipykernel_1517931/4035410637.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['Nat_Synth'] = pd.Categorical(annotation['Nat_Syn'])


['E8.5']
Categories (1, object): ['E8.5']

In [16]:
new_names = adata.var_names.to_frame(name='gene_short_name').merge(db, on='gene_short_name', how='inner')
new_names.drop_duplicates(subset='gene_short_name', inplace=True)

adata = adata[:, adata.var_names.isin(list(new_names['gene_short_name']))]

adata.var_names = new_names['gene_ID']
adata.var_names

Index(['ENSMUSG00000051951', 'ENSMUSG00000089699', 'ENSMUSG00000102343',
       'ENSMUSG00000025900', 'ENSMUSG00000025902', 'ENSMUSG00000104328',
       'ENSMUSG00000033845', 'ENSMUSG00000025903', 'ENSMUSG00000104217',
       'ENSMUSG00000033813',
       ...
       'ENSMUSG00000061654', 'ENSMUSG00000079834', 'ENSMUSG00000079222',
       'ENSMUSG00000062783', 'ENSMUSG00000096808', 'ENSMUSG00000079800',
       'ENSMUSG00000079808', 'ENSMUSG00000095041', 'ENSMUSG00000096730',
       'ENSMUSG00000095742'],
      dtype='object', name='gene_ID', length=22327)

In [17]:
adata.obs.stage.unique()

['E8.5']
Categories (1, object): ['E8.5']

In [18]:
adata_nat = adata[adata.obs['Nat_Synth']=='Natural']
adata_nat

View of AnnData object with n_obs × n_vars = 26948 × 22327
    obs: 'Nat_Synth', 'embryo_id', 'celltype', 'stage'

In [19]:
adata_synth = adata[adata.obs['Nat_Synth']=='Synthetic']
adata_synth

View of AnnData object with n_obs × n_vars = 40657 × 22327
    obs: 'Nat_Synth', 'embryo_id', 'celltype', 'stage'

In [21]:
adata_synth.obs.stage.unique()

['E8.5']
Categories (1, object): ['E8.5']

In [22]:
adata_nat.write('/home/icb/jonas.flor/gastrulation_atlas/data/invito_gastrulation.h5ad')
adata_synth.write('/home/icb/jonas.flor/gastrulation_atlas/data/exvito_gastrulation.h5ad')